# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import gkey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = "Output/Weather_Data.csv"
weather_df = pd.read_csv(file_path)
weather_df

,City,Country,Latitude,Longitude,Max Temp (F),Humidity %,Cloudiness %,Wind Speed (mph),Date
0,Yining,CN,29.04,114.56,55.92,89,99,1.03,1603054376
1,Lagoa,PT,39.05,-27.98,62.89,73,54,21.41,1603054377
2,Nacala,MZ,-14.54,40.67,71.55,87,0,3.20,1603054378
3,Puerto Ayora,EC,-0.74,-90.35,80.60,65,20,23.04,1603054380
4,Tarakan,ID,3.30,117.63,79.70,85,100,7.72,1603054382
...,...,...,...,...,...,...,...,...,...
553,Çeşme,TR,38.32,26.31,62.60,77,20,1.12,1603055139
554,Buraidah,SA,26.33,43.98,73.40,23,0,4.70,1603055140
555,Kieta,PG,-6.22,155.63,81.39,75,84,1.39,1603055142
556,Kungurtug,RU,50.60,97.52,30.06,84,97,1.81,1603055143


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=gkey)

In [4]:
 # Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Store humidity % in humidity
humidity = weather_df["Humidity %"]

# Set max intensity for humidity
max_intensity = weather_df["Humidity %"].max()

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 max_intensity=max_intensity,
                                 point_radius=25)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
hotel_df = weather_df.loc[weather_df["Max Temp (F)"] <= 80]
hotel_df = hotel_df.loc[hotel_df["Max Temp (F)"] >= 70]
hotel_df = hotel_df.loc[hotel_df["Wind Speed (mph)"] < 7]
hotel_df = hotel_df.loc[hotel_df["Cloudiness %"] == 0]
hotel_df

,City,Country,Latitude,Longitude,Max Temp (F),Humidity %,Cloudiness %,Wind Speed (mph),Date
2,Nacala,MZ,-14.54,40.67,71.55,87,0,3.20,1603054378
24,Opuwo,NaN,-18.06,13.84,74.16,19,0,5.68,1603054407
33,Bandarbeyla,SO,9.49,50.81,79.05,81,0,3.91,1603054419
108,Salalah,OM,17.02,54.09,78.80,34,0,4.70,1603054518
124,Senanga,ZM,-16.12,23.27,74.32,20,0,5.35,1603054541
173,Poya,NC,-21.35,165.15,72.54,61,0,5.75,1603054614
179,General Roca,AR,-39.03,-67.58,77.00,23,0,5.82,1603054624
183,Warri,NG,5.52,5.75,76.75,90,0,3.31,1603054628
242,Bulawayo,ZW,-20.15,28.58,73.27,29,0,6.96,1603054477
335,Safaga,EG,26.73,33.94,78.80,61,0,4.70,1603054846


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Add Hotel Name column to data frame
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Max Temp (F),Humidity %,Cloudiness %,Wind Speed (mph),Date,Hotel Name
2,Nacala,MZ,-14.54,40.67,71.55,87,0,3.20,1603054378,
24,Opuwo,NaN,-18.06,13.84,74.16,19,0,5.68,1603054407,
33,Bandarbeyla,SO,9.49,50.81,79.05,81,0,3.91,1603054419,
108,Salalah,OM,17.02,54.09,78.80,34,0,4.70,1603054518,
124,Senanga,ZM,-16.12,23.27,74.32,20,0,5.35,1603054541,


In [8]:
# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "lodging",
    "key": gkey
}

# Use the latitude and longitude to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Try to find Hotel Name for each city.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

# Display data frame with hotel names
hotel_df

,City,Country,Latitude,Longitude,Max Temp (F),Humidity %,Cloudiness %,Wind Speed (mph),Date,Hotel Name
2,Nacala,MZ,-14.54,40.67,71.55,87,0,3.20,1603054378,Nacala Plaza Business Design Hotel
24,Opuwo,NaN,-18.06,13.84,74.16,19,0,5.68,1603054407,Hotel Pension Le Manoir
33,Bandarbeyla,SO,9.49,50.81,79.05,81,0,3.91,1603054419,REER XOOSHLE GROUP
108,Salalah,OM,17.02,54.09,78.80,34,0,4.70,1603054518,Hilton Salalah Resort
124,Senanga,ZM,-16.12,23.27,74.32,20,0,5.35,1603054541,Christopher Nasilele House
173,Poya,NC,-21.35,165.15,72.54,61,0,5.75,1603054614,Sheraton New Caledonia Deva Spa & Golf Resort
179,General Roca,AR,-39.03,-67.58,77.00,23,0,5.82,1603054624,Hotel del Comahue
183,Warri,NG,5.52,5.75,76.75,90,0,3.31,1603054628,BON Hotel Delta
242,Bulawayo,ZW,-20.15,28.58,73.27,29,0,6.96,1603054477,4 On Housman Backpackers
335,Safaga,EG,26.73,33.94,78.80,61,0,4.70,1603054846,Serenity Makadi Beach


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.symbol_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))